## Agreements

In [1]:
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [2]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
    if agg['user__username__count'] >= 9:
         users.append(agg['user__username'])   
users

['amoro', 'jmperez', 'jzajac', 'mkondra', 'ndebandi']

In [3]:

base_query = {
    "user__username__in": users
}
ArticleLabel.objects.filter(**base_query).count()

100

¿Cuántos etiquetamos todes?

In [4]:
import pandas as pd

pd.options.display.max_columns = 40


articles_labelled = {label.article_id for label in ArticleLabel.objects.filter(**base_query)}

df = pd.DataFrame({"name":users})
df.set_index("name", inplace=True)

for label in ArticleLabel.objects.filter(**base_query):
    username = label.user.username
    df.loc[username, label.article.title] = int(label.is_interesting)
    
df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?,"Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”",Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo,Intentó violar a su novia y ella lo asesinó a escobazos,Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”,"Solo el 35,5% de la población aprueba la gestión de la pandemia realizada por el Gobierno",Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”,"Según un experto italiano, “el coronavirus se está debilitando y podría morir solo sin una vacuna”",Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño,"Filminas para todos, menos para Maduro",Ginés González: “La semana que viene aviones de Aerolíneas irán a China a buscar insumos de bioseguridad”,Florencia Peña se diferenció de los argentinos que se quieren ir del país: “Muchos queremos seguir apostando”,Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos,Escándalo en España: fallan los tests de coronavirus fabricados en China,"""Evolucionen"": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero","Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina"""
name,,,,,,,,,,,,,,,,,,,,,,,,,
amoro,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
jzajac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ndebandi,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Busquemos los que están etiquetados por todos

In [5]:
df = df.loc[["jmperez", "jzajac", "mkondra"]]

df = df[df.columns[df.notna().all()]]

df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?,"Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”",Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo,Intentó violar a su novia y ella lo asesinó a escobazos,Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”,"Solo el 35,5% de la población aprueba la gestión de la pandemia realizada por el Gobierno",Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”,"Según un experto italiano, “el coronavirus se está debilitando y podría morir solo sin una vacuna”",Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño,"Filminas para todos, menos para Maduro",Ginés González: “La semana que viene aviones de Aerolíneas irán a China a buscar insumos de bioseguridad”,Florencia Peña se diferenció de los argentinos que se quieren ir del país: “Muchos queremos seguir apostando”,Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos,Escándalo en España: fallan los tests de coronavirus fabricados en China,"""Evolucionen"": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero","Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina"""
name,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
jzajac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [6]:
from nltk import agreement

coders = df.loc[["jzajac", "jmperez", "mkondra"]].values

formatted_labels = []
for i in range(coders.shape[0]):
    # Acá las ponemos siguiendo el siguiente formato
    # [num etiquetador, num instancia, label]
    for j in range(coders.shape[1]):
        formatted_labels.append((i, j, coders[i, j]))
        

ratingtask = agreement.AnnotationTask(data=formatted_labels)

ratingtask.kappa()

0.11645847769732925

Ergh...bueno

## Agreements sobre odio

Ok, acá agarremos todos los labels sobre comentarios de todes

In [7]:
users = [
    "jmperez", 
    "mkondra", 
    "ndebandi",
    "jzajac",
    "amoro",
]

article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)

formatted_labels = []

df_comments = pd.DataFrame({"name": users})

df_comments.set_index("name", inplace=True)

for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        df_comments.loc[username, comment_label.comment_id] = comment_label.is_hateful

# Saco los que no estén etiquetados por todes
#df_comments = df_comments[df_comments.columns[df_comments.notna().all()]]

df_comments

,311633,311634,311635,311636,311637,311638,311639,311640,311641,311642,311643,311644,311645,311646,311647,311648,311649,311650,311651,311652,...,311844,311845,311846,311847,311848,311849,311850,311851,311852,311853,311854,311855,311856,311857,311858,311859,311860,311861,311862,311863
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mkondra,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
ndebandi,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jzajac,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
amoro,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
from nltk import agreement

def kappa_of(df, subset=None):
    if subset:
        coders = df.loc[subset]
    else:
        coders = df
    
    coders = coders[coders.columns[coders.notna().all()]]
    
    coders = coders.values
    formatted_labels = []
    for i in range(coders.shape[0]):
        # Acá las ponemos siguiendo el siguiente formato
        # [num etiquetador, num instancia, label]
        for j in range(coders.shape[1]):
            formatted_labels.append((i, j, coders[i, j]))


    ratingtask = agreement.AnnotationTask(data=formatted_labels)

    return ratingtask.kappa(), coders.shape[1]


for subset in [
    users,
    [u for u in users if u != "jzajac"],
    ["jmperez", "mkondra", "ndebandi", "jzajac"],
    ["jmperez", "mkondra", "ndebandi"],
    ["jmperez", "mkondra", "jzajac"],
    ["jmperez", "mkondra", "amoro"],
    ["mkondra", "ndebandi", "jzajac"],
    ]:
    name = ' '.join(subset)
    kappa, instances = kappa_of(df_comments, subset)
    print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")
    
print("\n\nPares\n\n")

for i in range(len(users)):
    for j in range(i+1, len(users)):
        subset = [users[i], users[j]]
        name = ' '.join(subset)
        kappa, instances = kappa_of(df_comments, subset)
        print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")


Fleiss K para jmperez mkondra ndebandi jzajac amoro    = 0.435 (#instancias = 251)
Fleiss K para jmperez mkondra ndebandi amoro           = 0.537 (#instancias = 251)
Fleiss K para jmperez mkondra ndebandi jzajac          = 0.414 (#instancias = 273)
Fleiss K para jmperez mkondra ndebandi                 = 0.526 (#instancias = 273)
Fleiss K para jmperez mkondra jzajac                   = 0.466 (#instancias = 688)
Fleiss K para jmperez mkondra amoro                    = 0.547 (#instancias = 251)
Fleiss K para mkondra ndebandi jzajac                  = 0.417 (#instancias = 521)


Pares


Fleiss K para jmperez mkondra                          = 0.616 (#instancias = 688)
Fleiss K para jmperez ndebandi                         = 0.438 (#instancias = 273)
Fleiss K para jmperez jzajac                           = 0.443 (#instancias = 688)
Fleiss K para jmperez amoro                            = 0.421 (#instancias = 251)
Fleiss K para mkondra ndebandi                         = 0.542 (#instancias =

In [12]:
print("Cantidad de mensajes etiquetados como odiosos")
df_comments.sum(axis=1)

Cantidad de mensajes etiquetados como odiosos


name
jmperez     204.0
mkondra     165.0
ndebandi     96.0
jzajac      481.0
amoro        84.0
dtype: float64

## Agreements sobre cada categoría

In [13]:
users = [
    "jmperez", 
    "mkondra", 
    "ndebandi",
    "jzajac",
    "amoro",
]


idx = pd.MultiIndex.from_product([CommentLabel.type_mapping, users],
                                names=["categoria", "etiquetador"])

df_labels = pd.DataFrame(index=idx)

df_labels

Empty DataFrame
Columns: []
Index: [(MUJER, jmperez), (MUJER, mkondra), (MUJER, ndebandi), (MUJER, jzajac), (MUJER, amoro), (LGBTI, jmperez), (LGBTI, mkondra), (LGBTI, ndebandi), (LGBTI, jzajac), (LGBTI, amoro), (RACISMO, jmperez), (RACISMO, mkondra), (RACISMO, ndebandi), (RACISMO, jzajac), (RACISMO, amoro), (POBREZA, jmperez), (POBREZA, mkondra), (POBREZA, ndebandi), (POBREZA, jzajac), (POBREZA, amoro), (POLITICA, jmperez), (POLITICA, mkondra), (POLITICA, ndebandi), (POLITICA, jzajac), (POLITICA, amoro), (DISCAPACIDAD, jmperez), (DISCAPACIDAD, mkondra), (DISCAPACIDAD, ndebandi), (DISCAPACIDAD, jzajac), (DISCAPACIDAD, amoro), (ASPECTO, jmperez), (ASPECTO, mkondra), (ASPECTO, ndebandi), (ASPECTO, jzajac), (ASPECTO, amoro), (CRIMINAL, jmperez), (CRIMINAL, mkondra), (CRIMINAL, ndebandi), (CRIMINAL, jzajac), (CRIMINAL, amoro), (OTROS, jmperez), (OTROS, mkondra), (OTROS, ndebandi), (OTROS, jzajac), (OTROS, amoro)]

In [14]:

article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)


for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        for name, field_name in CommentLabel.type_mapping.items():
            df_labels.loc[(name, username), comment_label.comment_id] = getattr(comment_label, field_name)

df_labels

311633 311634 311635 311636 311637 311638 311639  \
categoria    etiquetador                                                    
MUJER        jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
LGBTI        jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
RACISMO      jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
POBREZA      jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
POLITICA     jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
DISCAPACIDAD jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
ASPECTO      jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
CRIMINAL     jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   
OTROS        jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac       False  False  False  False  False  False  False   
             amoro        False  False  False  False  False  False  False   

                         311640 311641 311642 311643 311644 311645 311646  \
categoria    etiquetador                                                    
MUJER        jmperez        NaN    NaN    NaN    NaN    NaN    NaN    NaN   
             mkondra      False  False  False  False  False  False  False   
             ndebandi     False  False  False  False  False  False  False   
             jzajac 

In [15]:
print("##", ' - '.join(users), '\n')

for category in CommentLabel.type_mapping:
    try:
        kappa, instances = kappa_of(df_labels.loc[category].loc[["amoro", "mkondra"]])
        print(f"| {category:<12}   |  {kappa:.3f}")
    except ZeroDivisionError as e:
        print(f"No hay elementos para calcular Kappa de {category} - salteando")

print(f"Instancias: {instances}")


## jmperez - mkondra - ndebandi - jzajac - amoro 

| MUJER          |  0.517
No hay elementos para calcular Kappa de LGBTI - salteando
| RACISMO        |  0.498
| POBREZA        |  -0.003
| POLITICA       |  0.000
| DISCAPACIDAD   |  0.127
| ASPECTO        |  0.150
| CRIMINAL       |  0.755
| OTROS          |  0.000
Instancias: 499




### jmperez - mkondra - jzajac 

| Category      | Agreement |
|---------------|-----------|
| MUJER        |  0.461    |
| LGBTI        |  0.645    |
| RACISMO      |  0.600    |
| POBREZA      |  0.387    |
| POLITICA     |  0.309    |
| DISCAPACIDAD | 0.253     |
| ASPECTO      |  0.512    |
| CRIMINAL     |  0.728    |
| OTROS        |  -0.002   |
Instancias: 688

### mkondra - jzajac 
| Category      | Agreement |
|---------------|-----------|
| MUJER       |  0.303    |
| LGBTI       |  0.503    |
| RACISMO     |  0.510    |
| POBREZA     |  0.364    |
| POLITICA    |  0.154    |
| DISCAPACIDAD|  0.098    |
| ASPECTO     |  0.460    |
| CRIMINAL    |  0.669    |
| OTROS       |  -0.004   |

Instancias: 986

### jmperez - mkondra 
| Category      | Agreement |
|---------------|-----------|
| MUJER          |  0.601   |   
| LGBTI          |  0.754   |
| RACISMO        |  0.715   |
| POBREZA        |  0.230   |
| POLITICA       |  0.289   |
| DISCAPACIDAD   |  0.758   |
| ASPECTO        |  0.699   |
| CRIMINAL       |  0.850   |
| OTROS          |  0.000   |
Instancias: 688

### jmperez - mkondra - ndebandi - jzajac 

| Category      | Agreement |
|---------------|-----------|
| MUJER         |     0.386 |
| RACISMO       |     0.541 |
| POBREZA       |     0.202 |
| POLITICA      |     0.308 |
| DISCAPACIDAD  |     0.280 |
| ASPECTO       |     0.606 |
| CRIMINAL      |     0.730 |

Instancias: 273

## MUJER


In [16]:
from collections import defaultdict

users = [
    #"jmperez",
    "mkondra",
    "jzajac",
]

def no_null_columns(df):
    return df[df.columns[df.notna().all()]]

            
def show_from_df(df, show=False):
    article_to_comments = defaultdict(list)
    
    for comment_id in df.columns:
        comment = Comment.objects.get(id=comment_id)
        article_to_comments[comment.article_id].append(comment)
        
    for article_id, comments in article_to_comments.items():
        print("="*80 + "\n")
        article = comments[0].article
        print(article.title, "\n")
        
        for comment in comments:
            print(comment.text, end="")
        
            if show:
                print(f" {dict(df[comment_id])}")
            else:
                print("")

In [17]:
def get_agrees_and_disagrees(df_labels, category, users):    
    category_df = df_labels.loc[category].loc[users]

    category_df = no_null_columns(category_df)
    agree_comments = category_df[category_df.columns[category_df.all()]]
    no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
    disagree_comments = category_df[category_df.columns[no_agree]]
    
    return agree_comments, disagree_comments


agree_comments, disagree_comments = get_agrees_and_disagrees(df_labels, "MUJER", users)

## Ejemplos de odio:

Como regla general, si un comentario ofensivo podría ser usado casi indistintamente para atacar a un hombre, entonces no debería ser considerado discriminatorio contra las mujeres


In [18]:
%%html

<blockquote class="twitter-tweet"><p lang="es" dir="ltr">Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” <a href="https://t.co/0yLfUkUacL">https://t.co/0yLfUkUacL</a></p>&mdash; infobae (@infobae) <a href="https://twitter.com/infobae/status/1296507657189314566?ref_src=twsrc%5Etfw">August 20, 2020</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>



### Ejemplos discriminatorios:

- La marcha fue p q dejen d avasallar los derechos y la democracia y p defender la República d esta panda d ladrones y enchufados ineptos y faltos d talento como esta **rubia tarada** q actúa igual hace 40 años! No a la reforma judicial y demás atropellos!

- No me sorprende toda la vida hizo el papel de la pelotuda con gomas
- Pero callate si te acostabas con Fito! 

- Miauuuuu 

### Ejemplos no discriminatorios (aunque posiblemente ofensivos):
- *No existió ninguna marcha anticuarentena....boba* **(podría decírsele a un hombre también)**
- *RECORDAS KUANDO VENDIAS JUNTO A FITO KILOS DE MERKA ????*
- *Otra forra* **(agresivo pero a priori no misógino)**
- *Y a quién le importa?* **agresivo, no misógino**
- *Vieja merquera* **(la discrimina por vieja, por adicta, pero no por mujer)**
- *No se ni quien es, pero sin conocerla puedo decir que es una estúpida. Irresponsabilidad es lo que esta haciendo este gobierno fascista con el pueblo* 

### Ejemplos a revisar

- *Y vos un foco infeccioso después de comerte al murciélago de Feto 🤮🤮🤮🤮*
- *De la mía no vas a comer nunca descerebrada, hasta las publicidades en las q apareces cambio.*
- *Cara gorda,cara flaca,igual cara de traste*
- *Y a quién choto le puede importar la opinión de ésta mina ?*


In [19]:
%%html
<blockquote class="twitter-tweet"><p lang="es" dir="ltr">&quot;Evolucionen&quot;: Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero <a href="https://t.co/DDtMPp6OgZ">https://t.co/DDtMPp6OgZ</a> <a href="https://t.co/HH78Akp97Y">pic.twitter.com/HH78Akp97Y</a></p>&mdash; LA NACION (@LANACION) <a href="https://twitter.com/LANACION/status/1273676406614978561?ref_src=twsrc%5Etfw">June 18, 2020</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

In [20]:
show_from_df(agree_comments)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION @Sonia43917925 Cuál marcha anti cuarentena? 
La marcha fue p q dejen d avasallar los derechos y la democracia y p defender la República d esta panda d ladrones y enchufados ineptos y faltos d talento como esta rubia tarada q actúa igual hace 40 años!
No a la reforma judicial y demás atropellos!
@LANACION Chuparle la pija a Fito Páez fue una irresponsabilidad sanitaria, payasa. De qué hablás??
@LANACION Le prestaste la cachucha a Feto Paez Roth!!!🤮🤮 eso si que es una "irresponsabilidad sanitaria"
@LANACION Otra imbesil kaka! Anda hacerte ortear!

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

@infobae Irresponsabilidad es la doble vara del colectivo de actrices para con casos como el de Alperovich. 
Y encima ¿¿¿ justo esta lustradora de tararira de Feto habla de irresponsabilidad???
@infobae Mira quien habla, la que le chupaba la p*ja a Feto Páez
@infob

Busquemos dónde hay discrepancias

In [21]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Que hace esta mina o sra no se {'mkondra': False, 'jzajac': True}
@LANACION Quien es esta vieja del agua? {'mkondra': False, 'jzajac': True}
@LANACION A mí vos me pareces una pelotuda importante {'mkondra': False, 'jzajac': True}
@LANACION Ahora puedo dormir tranquilo. La científica infectóloga y doctora en medicina, Cecilia Roth, dijo la verdad de la milanesa. {'mkondra': False, 'jzajac': True}
@LANACION Cómo el botox en tu cara {'mkondra': False, 'jzajac': True}
@LANACION Y vos un foco infeccioso después de comerte al murciélago de Feto 🤮🤮🤮🤮 {'mkondra': False, 'jzajac': True}
@LANACION Ay chechu que malll ahora no voy a poder dormir a la noche 😫 {'mkondra': False, 'jzajac': True}
@LANACION ¿Y a quién le interesa lo que diga esta mujer? ¿Es científica? {'mkondra': False, 'jzajac': True}
@LANACION No existió ninguna marcha anticuarentena....boba {'mkondra': False, 'jzajac': True}
@LANACION 

## LGBTI


In [22]:
category_df = df_labels.loc["LGBTI"].loc[users]

category_df = no_null_columns(category_df)
agree_comments = category_df[category_df.columns[category_df.all()]]
agree_comments.shape

no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
no_agree_comments = category_df[category_df.columns[no_agree]]

In [23]:
show_from_df(agree_comments)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

@LANACION Novarecio puto gorilon.
@LANACION Viejo maricota
@LANACION Qué payasa ...
@LANACION Aféitense, chicas! 💋
@LANACION Que par de putos.
@LANACION 2 trolos casados por una rabina y escoltados por 2 pavorreales con la camiseta de river
@LANACION Uno mas idiota que el otro manga de t...
@LANACION En menos de un año se separan, como todos los trolos que se casan. Es fija.
@LANACION Asaaaaaaa ahora entiendo porque siempre está tan histérica Luisitp el novio no se la pone 😀😀😀😀😀
@LANACION Dos ascos
@LANACION Que asco de gente....


In [24]:
show_from_df(no_agree_comments, show=True)


Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público” 

@infobae Cerrá el orto payaso!!! {'mkondra': False, 'jzajac': True}

Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 

@clarincom Naa pero vayanse a cagar hijos de remil puta, y yo aca respetando la cuarentena como un boludo, me pueden chupar bien la pija ahora {'mkondra': False, 'jzajac': True}
@clarincom La única cuestión de imagen que les jode a ustedes es la de Brasil , Chile, Bolivia, y toda América al lado nuestro. Chupenla. {'mkondra': False, 'jzajac': True}

"Evolucionen": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero 

@LANACION Encima parece heterosexual! 
Sino puede entrar a cualquier lado en tetas que está bien.. {'mkondra': False, 'jzajac': True}

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

@LANAC

## RACISMO

In [25]:
category_df = df_labels.loc["RACISMO"].loc[users]

category_df = no_null_columns(category_df)
agree_comments = category_df[category_df.columns[category_df.all()]]
agree_comments.shape

no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
no_agree_comments = category_df[category_df.columns[no_agree]]

In [26]:
show_from_df(agree_comments)


Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos 

@infobae @SilviaDiluzio12 SACANLE LOS PLANES A TODOS LOS PARAGUAYOS CRUZANDO LA FRONTERA PARA VENIR A COBRAR.
PONGANLE UNA ESTAMPILLA EN EL CULO Y Q NO PUEDAN ENTRAR!! ES UNA VERGÜENZA!!!

¿Terminará Argentina como Venezuela? 

@infobae Si terminamos nosé, pero que los hay los hay son una plaga estos venezolanos a lo mejor toman el país

Escándalo en España: fallan los tests de coronavirus fabricados en China 

@infobae Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
@infobae @anavas4 No entendemos que lo CHINO no sirve🙄
@infobae Y son chinos cuando fueron fiables
@infobae Estos son socios del que hizo el virus!!!!
@infobae CHINOS 

In [27]:
show_from_df(no_agree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae Papá...que caripela!!! Menos mal que no murió un santo....sino mañana hablan de mártir y lo hacen beato. {'mkondra': False, 'jzajac': True}
@infobae Mira lo que es ese muñeco por favor 💀 {'mkondra': False, 'jzajac': True}

Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino 

@LANACION Maldito Tom Hanks! Aunque sabía q eran los barcos chicos y japoneses, nunca dude de su participación. {'mkondra': False, 'jzajac': True}
@LANACION Mucho pirata chino.  Quizás cambiaron mariscos por barbijos {'mkondra': False, 'jzajac': True}
@LANACION Chinos y coreanos, los primeros!! {'mkondra': False, 'jzajac': True}
@LANACION Y el precio del pescado es prohibitivo para el consumidor Argentino,hace unos años atrás la aviación Chilena ametralló a barcos ladrones Franceses qvenían a robarles la Langosta d mar,creo no aparecieron más y se les debería dar un ultimátum y a partir de ahí am

## POBREZA

In [28]:
category_df = df_labels.loc["POBREZA"].loc[users]

category_df = no_null_columns(category_df)
agree_comments = category_df[category_df.columns[category_df.all()]]
agree_comments.shape

no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
no_agree_comments = category_df[category_df.columns[no_agree]]

In [29]:
show_from_df(agree_comments)


El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana 

@clarincom Hola planeros
@clarincom ¡CLARÍN, BASTA DE MANTENER VAGOS CHORIPLANEROS COMO ESTOS ATORRANTES, CON MIS IMPUESTOS, QUE SE BUSQUEN UN TRABAJO DIGNO Y VAYAN A LABURAR, CARADURAS, DEJEN DE VIVIR DEL ESTADO!!!

Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino 

@LANACION y ustedes lo que roban no tiene nombre, planeros

Solo el 35,5% de la población aprueba la gestión de la pandemia realizada por el Gobierno 

@infobae Empleados estatales y planeros
@infobae T los come polenta tienen que morir con el relato.


In [30]:
show_from_df(no_agree_comments, show=True)


El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana 

@clarincom Es decir , le van a sacar guita a gente que llegó ahí trabajando para darle guita a los vagos de siempre? {'mkondra': False, 'jzajac': True}
@clarincom Se necesita más plata para aumentar planes, Ife, comedores, etc... {'mkondra': False, 'jzajac': True}

Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos 

@infobae Chás chás en la cola? Es un delito, ¿Por qué no detenerlos? {'mkondra': False, 'jzajac': True}
@infobae Y carcel! {'mkondra': False, 'jzajac': True}
@infobae Y q los pongan a trabajar..... pero no criemos vagos y analfabetos {'mkondra': False, 'jzajac': True}
@infobae Pero que ningún impulsen más nada en la provincia que se haga cargo Chucky el enano maldito que se ocupen de lo que está pasando a nivel nacional porque no se ocupan de hacer un proyecto para la gente que trabaja estoy harta de los 

## POLITICA

In [31]:
category_df = df_labels.loc["POLITICA"].loc[users]

category_df = no_null_columns(category_df)
agree_comments = category_df[category_df.columns[category_df.all()]]
agree_comments.shape

no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
no_agree_comments = category_df[category_df.columns[no_agree]]


In [32]:

show_from_df(agree_comments)


Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa 

@clarincom Pena de muerte deberían darle a ese enfermo mental comunista
@clarincom @VillaFernando_ Mira cachudo ladrón hpta eres noticia mundial @MashiRafael

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Obviamente,si sos la alcahuete number one de los corruptos!!

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

@infobae Hablo la acomodaticia. Zurda de country
@infobae Reventa bolchevique. Viva la libertad 🗽 nos vemos 👋
@infobae Una militante paga al servicio del poder de korrupto, chirolitas de #cfkladronadelanaciónargentina

Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino 

@LANACION y ustedes lo que roban no tiene nombre, planeros

Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos 

@infobae Y

**TODOS ESTOS FUERON jzajac TRUE Y mkondra FALSE**

In [33]:
show_from_df(no_agree_comments, show=False)


Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa 

@clarincom Que lastima que este pais no tengan huevos y suenten chorros @CorteSupremaJ
@clarincom Otro de los salvadores de pobres de la patria grande!!! Todos y todas ladrones!!!
@clarincom Cayó por fin este ladron. Dejo en la miseria a a Ecuador
@clarincom Tendrian que haberlo condenado de por vida,es un critino igual que todos los populistas, podre mas poder mas dinero por robo y corrupto hasta la medulla aca tenemos para regular.
@clarincom Los Korruptos están cayendo de a uno
@clarincom De a poco van todos los comunistas en cana, luego le tocará a Cristina Kishner, Mujica, Maduro, Evo y cia.
@clarincom Al fin justicia, este delincuente nos gobernó 10 años
@clarincom Ahi esta el socialismos señoressss... Todos la misma. M...!!!!
@clarincom @Cris_noticias Otro hdp q se va a venir a refugiar a nustro corrupto pais
@clarincom Ahora faltan los demás socialistas amigos de Correa que están en Lati

## ASPECTO

In [34]:
CommentLabel.type_mapping.keys()

dict_keys(['MUJER', 'LGBTI', 'RACISMO', 'POBREZA', 'POLITICA', 'DISCAPACIDAD', 'ASPECTO', 'CRIMINAL', 'OTROS'])

In [35]:
category_df = df_labels.loc["ASPECTO"].loc[users]

category_df = no_null_columns(category_df)
agree_comments = category_df[category_df.columns[category_df.all()]]
agree_comments.shape

no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
no_agree_comments = category_df[category_df.columns[no_agree]]

In [36]:
show_from_df(agree_comments)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Quien es esta vieja del agua?

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

@infobae Cara gorda,cara flaca,igual cara de traste.
@infobae El kirchnerismo te afea. Comprobación nro. 335

Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público” 

@infobae @SergioBerniArg Ahh perfecto pelotudo ¿ y como se supone que yo vaya a laburar si no tengo transporte publico al igual que muchos otros? Viejo pelotudo. Anda a seguir jugando con la metralleta finjiendo ser Rambo, mogolico.

Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos” 

@infobae Pobre viejo...
@infobae Sos un payaso. Jubilate
@infobae Muchachos devuélvanlo al geriátrico que la hora del recreo ya paso. No sabía de la existencia de protocolo para pelotudos y suicidas.

**Todos los ejemplos son sólo marcadas por Joaquín**

In [37]:
show_from_df(no_agree_comments)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Te comiste a Feto Paez irresponsable‼
@LANACION Cómo el botox en tu cara
@LANACION Y vos un foco infeccioso después de comerte al murciélago de Feto 🤮🤮🤮🤮

El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae Papá...que caripela!!! Menos mal que no murió un santo....sino mañana hablan de mártir y lo hacen beato.
@infobae Mira lo que es ese muñeco por favor 💀
@infobae Un arma tatuada en la cara... Uno menos como dicen muchos...
@infobae Una lacra menos 🤷🏼‍♂️
@infobae Delicatesen los tatus

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

@infobae Vieja merquera
@infobae solo entré para ver como te puteaban 500 veces..reventada
@infobae La cara de alegría...mamita 🤦‍♂️
@infobae Vieja HDP
@infobae La irresponsabilidad sanitaria es la cantidad de cirugías estéticas q le hicieron en la cara, HORRIBLE, son un atentado

Al

## CRIMINAL

In [38]:
category_df = df_labels.loc["CRIMINAL"].loc[users]

category_df = no_null_columns(category_df)
agree_comments = category_df[category_df.columns[category_df.all()]]
agree_comments.shape

no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
no_agree_comments = category_df[category_df.columns[no_agree]]


In [39]:
show_from_df(agree_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae Loa beneficios del Covid19
@infobae Un respirador menos
@infobae Uno menos!
@infobae Que no pare el fuego y rociarlo con nafta es una buena receta..
@infobae Uno menos 👏
@infobae Uno menos 👏🏽👏🏽
@infobae 👏🏻👏🏻👏🏻 sigan sigan
@infobae Que linda noticia....perfecto!!!👏👏👏
@infobae No se pierde nada son una mierda . Todos los que están presos... que mueran todos
@infobae Parece que este virus nos va a depurar un poco de algunos males...
@infobae Napal y tema solucionado
@infobae Esperemos que sea el primero pero no el último
@infobae Uno menos
@infobae Uno menos..
@infobae Solo Me preocupa por los gastos q ocasionaron. Hay q ir con todo
@infobae Uno menos
@infobae Unomenos
@infobae Uno menos! De los malos
@infobae no se perdió nada
@infobae Al fin una buena!!!
@infobae Pero miren a esta criatura de belcebú, con la cría de asesino en brazos para dar lástima. Ay! ay! Cobras y leones hambrientos" Proble

In [40]:
show_from_df(no_agree_comments, show=False)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Vos sos irresponsable social apoyando la dictadura kirchnerista. Desaparecen chicos todos los días, la policía está desbocada, los delincuentes liberados vuelven a delinquir , nos roban desde el gobierno con las compras... sos nada Roth, sólo fuiste.

El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae y la verdad que mucha lastima no me da..
@infobae Que lastima
@infobae Excelente
@infobae Podrían experimentar con estos seres posibles mutaciones y vacunas así hacen su aporte para la sociedad
@infobae Qué pérdida!
@infobae No perdimos nada entonces, palo y a la bolsa!!
@infobae Igual no servía para nada......
@infobae Subir la foto con un bebé no la hace menos culpable!
@infobae Pobre ciudadano, seguramente era inocente.
@infobae Papá...que caripela!!! Menos mal que no murió un santo....sino mañana hablan de mártir y lo hacen beato.
@infobae Una pena

## OTROS

In [41]:
category_df = df_labels.loc["OTROS"].loc[users]

category_df = no_null_columns(category_df)
agree_comments = category_df[category_df.columns[category_df.all()]]
agree_comments.shape

no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
no_agree_comments = category_df[category_df.columns[no_agree]]

In [42]:
show_from_df(agree_comments)

In [43]:
show_from_df(no_agree_comments, show=True)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Nadie la juzga por su adicción a las drogas. Que no venga a criticar las expresiones de voluntad. {'mkondra': False, 'jzajac': True}
@LANACION Simplemente no me interesa lo que diga una adicta {'mkondra': False, 'jzajac': True}
@LANACION Esta señora grupo de riesgo debería cuidarse de no excederse con la fafafa en vez de usar el dedito acusador. {'mkondra': False, 'jzajac': True}
@LANACION RECORDAS KUANDO VENDIAS JUNTO A FITO KILOS DE MERKA ???? {'mkondra': False, 'jzajac': True}

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

@infobae Vieja merquera {'mkondra': False, 'jzajac': True}
@infobae Irresponsabilidad sanitaria es vivir falopeada, Cecilia. {'mkondra': False, 'jzajac': True}

Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público” 

@infobae Propongo gueto total a la provincia de bueno

## LLAMADO A LA ACCIÓN

Queremos ver:

- Aquellas que hayan sido etiquetadas como odiosas


In [65]:
my_users = ['amoro', 'jmperez', 'jzajac', 'mkondra', 'ndebandi']
df_calls_for_action = pd.DataFrame({"name": my_users})
df_calls_for_action.set_index("name", inplace=True)

for comment_label in CommentLabel.objects.filter(is_hateful=True, article_label__user__username__in=my_users):
    username = comment_label.article_label.user.username
    df_calls_for_action.loc[username, comment_label.comment_id] = comment_label.calls_for_action

In [66]:
kappa_of(df_calls_for_action, ['amoro', 'jmperez', 'jzajac', 'mkondra', 'ndebandi'])

(0.266284257751689, 36)

In [67]:
for i in range(len(df_calls_for_action)):
    user1 = df_calls_for_action.index[i]
    for j in range(i+1, len(df_calls_for_action)):
        user2 = df_calls_for_action.index[j]
        
        print(f"Kappa {user1:<10} - {user2:<10} = {kappa_of(df_calls_for_action, [user1, user2])}")

Kappa amoro      - jmperez    = (0.1608876560332872, 55)
Kappa amoro      - jzajac     = (0.339597315436242, 82)
Kappa amoro      - mkondra    = (-0.04819277108433673, 58)
Kappa amoro      - ndebandi   = (0.2285714285714288, 60)
Kappa jmperez    - jzajac     = (0.450472667488697, 191)
Kappa jmperez    - mkondra    = (0.16836262719703984, 124)
Kappa jmperez    - ndebandi   = (0.2723404255319149, 57)
Kappa jzajac     - mkondra    = (0.36821192052980173, 159)
Kappa jzajac     - ndebandi   = (0.5074235807860265, 94)
Kappa mkondra    - ndebandi   = (0.2587859424920122, 58)


In [68]:
kappa_of(df_calls_for_action, ['mkondra', 'ndebandi'])

(0.2587859424920122, 58)

In [69]:
kappa_of(df_calls_for_action, ['jzajac', 'mkondra',])

(0.36821192052980173, 159)

Bajo, bajo... aunque podría ser peor

¿Sobre todos los comentarios etiquetados, qué proporción "llaman a la acción"?

In [87]:
df_calls_for_action.sum(axis=1) / df_calls_for_action.notna().sum(axis=1)

name
amoro       0.059524
jmperez     0.191176
jzajac      0.108108
mkondra     0.030303
ndebandi    0.104167
dtype: float64

In [88]:
df = no_null_columns(df_calls_for_action.loc[["jzajac", "mkondra"]])

df.sum(axis=1)

name
jzajac     20.0
mkondra     5.0
dtype: float64

In [89]:

agree_comments = df[df.columns[df.all()]]
agree_comments.shape

no_agree = ~((df.sum() == 0) | (df.sum() == len(users)))
no_agree_comments = df[df.columns[no_agree]]

In [92]:
show_from_df(agree_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae Napal y tema solucionado
@infobae Pero si tienen k morir todos los presos

Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo 

@clarincom Nuestra oportunidad de matar esta bosta..
@clarincom Algun infectado que le tosa en la cara a esta mujer please.

Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa 

@clarincom Pena de muerte deberían darle a ese enfermo mental comunista


In [94]:
show_from_df(no_agree_comments, show=True)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Esta señora grupo de riesgo debería cuidarse de no excederse con la fafafa en vez de usar el dedito acusador. {'jzajac': True, 'mkondra': False}
@LANACION Otra imbesil kaka! Anda hacerte ortear! {'jzajac': True, 'mkondra': False}

El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae Que no pare el fuego y rociarlo con nafta es una buena receta.. {'jzajac': True, 'mkondra': False}
@infobae No se pierde nada son una mierda . Todos los que están presos... que mueran todos {'jzajac': True, 'mkondra': False}
@infobae Esperemos que sea el primero pero no el último {'jzajac': True, 'mkondra': False}
@infobae Solo Me preocupa por los gastos q ocasionaron. Hay q ir con todo {'jzajac': True, 'mkondra': False}
@infobae @Llucia880955901 Pq no dejan que se maten entre ellos,son lacras que no sirven para nada, hubieran aprovechado y limpiaban todo de esta mierda {'